In [1]:
%load_ext watermark
import pandas as pd
import numpy as np
from typing import Type, Optional, Callable
from typing import List, Dict, Union

from review_methods_tests import collect_vitals, find_missing, find_missing_loc_dates
from review_methods_tests import make_a_summary,combine_survey_files

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

from setvariables import *

import reportclass

import session_config

# surveys = combine_survey_files(survey_files)
codes = pd.read_csv(code_data).set_index("code")
beaches = pd.read_csv(beach_data).set_index("slug")
land_cover = pd.read_csv(land_cover_data)
land_use = pd.read_csv(land_use_data)
streets = pd.read_csv(street_data)
river_intersect_lakes = pd.read_csv(intersection_attributes)

# The survey data

Once the surveyor has entered the data for the observations, there are labels attached to each record that identify attributes of the survey location. These labels are used for grouping records or collecting additional information about the survey location or the object found. Once the labels are complete the data is ready for analysis.

## Observation data

This data is received from the client in two packets:

::::{grid} 1 2 2 2

:::{grid-item}

__Meta data :__ .JSON object or python dictionary

* user
* location name, lat and lon
* date
* length of survey\*\*
* area of survey\*\*
* weights (plastic, all)\*
* number of participants\*
* number of staff\*
* time in minutes to complete all tasks\*
* total number of objects found
:::

:::{grid-item}

__Object inventory :__ an array, four columns:

* location name
* date
* code
* quantity

:::

::::

\* Not all projects provides all the data. For detailed results contact the maintainer

\*\* The data is all formatted in pcs/m, this is set to the units used for reporting baseline values

### Adding features

Survey locations are predefined, or must be configured before entering data for a survey. The configuration process has several steps.

1. Identify the correct city and canton name
2. Identify the type of location: _river, lake, or park_
3. Identify the riverbassin or geographic area of the location
4. Calculate the land use ratios for the location
5. For lakes, determine the location and distance to river intersections
6. The plain english (french, german) description of the code/object

__Aggregated data__

The following codes are aggregated together unless otherwise specified:

* 'Gfoams': 'G81', 'G82', 'G83'
* 'Gfrags': 'G78', 'G79', 'G80', 'G75', 'G76', 'G77'
* 'Gcaps': 'G21', 'G22', 'G23', 'G24'

_The unaggregated data is available on the repo using the following file files from the `data/end_process/` directory:_

* slr.csv
* mcbp.csv
* iqaasl.csv
* after_may_2021.csv

## Important dates

The dates of the sampling campaigns are expanded to include the surveys that happened between large organized campaigns. The start and end dates are defined below.

__Attention!!__ The codes used for each survey campaign are different. Different groups organized and conducted surveys using the MLW protocol. The data was then sent to us.

__MCBP:__ November 2015 - November 2016. The initial sampling campaign. Fragmented plastics (Gfrags/G79/G78/G76) were not sorted by size. All unidentified hard plastic items were classified in this manner.

* start_date = 2015-11-15
* end_date = 2017-03-31

__SLR:__ April 2017 - May 2018. Sampling campaign by the WWF. Objects less than 2.5 cm were not counted.

* start_date = 2017-04-01
* end_date = 2020-03-31

__IQAASL:__ April 2020 - May 2021. Sampling campaign mandated by the Swiss confederation. Additional codes were added for regional objects.

* start_date = 2020-04-01
* end_date = 2021-05-31

__Plastock (not added yet):__ January 2022 - December 2022. Sampling campaign from the Association pour la Sauvegarde du Léman. Not all objects were counted, They only identified a limited number of objects.

## The current survey data:

In [2]:
surveys = session_config.collect_survey_data()

In [3]:
surveys.info()

<class 'pandas.core.frame.DataFrame'>
Index: 298848 entries, 0 to 88214
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   sample_id        298848 non-null  object 
 1   code             298848 non-null  object 
 2   quantity         298848 non-null  int64  
 3   pcs/m            298848 non-null  float64
 4   feature_name     298848 non-null  object 
 5   location         298848 non-null  object 
 6   parent_boundary  298848 non-null  object 
 7   length           298848 non-null  float64
 8   groupname        298848 non-null  object 
 9   city             298848 non-null  object 
 10  canton           298848 non-null  object 
 11  feature_type     298848 non-null  object 
 12  en               298848 non-null  object 
 13  fr               298848 non-null  object 
 14  de               298848 non-null  object 
 15  date             298848 non-null  object 
dtypes: float64(2), int64(1), object(13)
memory u

In [4]:
%watermark -a hammerdirt-analyst -co --iversions

Author: hammerdirt-analyst

conda environment: cantonal_report

pandas    : 2.0.3
numpy     : 1.25.2
matplotlib: 3.7.1

